In [1]:
%run visualisation.ipynb
%run read_graph.ipynb

import pixiedust

g = read_nba_full("../Scrapers/BasketballReferenceScraper/data/Players.csv",
                  "../Scrapers/BasketballReferenceScraper/data/Edges.csv",
                  True, True)

all11_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142,3134,1849,2145,1945,3283,3669,1446,718,1031,1898,2577,2492,3502,620,3079,3672,1995,3383,4609,4090,65,2930,1504]
nba11_20 = g.subgraph(all11_20)

all15_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159, 1038, 1113, 1929, 3429, 4432, 1125, 863, 1588, 2586,4198,115,2925,1459,2235,1794,415,1460,564,4168,2384,1142]
nba15_20 = g.subgraph(all15_20)

all19_20 = [4524, 4418, 4778, 112, 3876, 1223, 2591, 3892, 614, 2889, 25, 4147, 3726, 1717, 1085, 2065, 2513, 955, 3311, 2542, 2938, 2004, 2177, 1517, 401, 2009, 3201, 266, 1620, 4393, 4396, 3719, 921, 1149, 1475, 4227, 44, 4278, 3159]
nba19_20 = g.subgraph(all19_20)

Pixiedust database opened successfully


Reading full NBA graph, begin...
Reading file ../Scrapers/BasketballReferenceScraper/data/Players.csv, begin...
Data lines read: 4800
Reading file ../Scrapers/BasketballReferenceScraper/data/Edges.csv, begin...
Data lines read: 146369
Reading NBA vertices, begin...
Vertices read: 4800
Reading NBA edges, begin...
Weights loaded correctly
Edges read: 146369
Loading succesfull


In [2]:
def build_lpa(graph, output):
    result = graph.community_label_propagation()
    resultArray = result.membership
    i = 0
    for v in graph.vs:
        v["communities"] = [resultArray[i]]
        i += 1

    plot_full(graph, output, 1)

In [3]:
build_lpa(nba11_20, "all_stars_1120.svg")
build_lpa(nba15_20, "all_stars_1520.svg")
build_lpa(nba19_20, "all_stars_1920.svg")

Plotting graph with 83 vertices and 315 edges, begin...
Plotting sucessful
Plotting graph with 60 vertices and 133 edges, begin...
Plotting sucessful
Plotting graph with 39 vertices and 38 edges, begin...
Plotting sucessful


In [60]:
def my_basic_lpa(graph, output):
    counters = []
    vertices_count = len(graph.vs)
    
    i = 0
    for v in graph.vs:
        v["communities"] = [i]
        i += 1
        counters.append(0)
    
    convergence = False
    iteration = 0
    while convergence == False and iteration < 20:
        iteration += 1
        convergence = True
        
        for idx in range(vertices_count):
            counters[idx] = 0
            
        for v in graph.vs:
            com = v["communities"][0]
            counters[com] += 1
            
        unique = sum(1 if counter > 0 else 0 for counter in counters)
        print("Iteration " + str(iteration) + ",communities " + str(unique))

        for v in graph.vs:
            current_community = v["communities"][0]   
            
            #clear counters
            for idx in range(vertices_count):
                counters[idx] = 0
            
            for e in v.all_edges():     
                if v == e.target_vertex and v == e.source_vertex:
                    print("Warn - out vertex equals current vertex")
                elif v != e.target_vertex:
                    com = e.target_vertex["communities"][0]
                    counters[com] += 1
                elif v != e.source_vertex:
                    com = e.source_vertex["communities"][0]
                    counters[com] += 1
                    
            if sum(counters) != v.degree():
                print("Warn: sum = " + str(sum(counters)) + " degree = " + str(v.degree()))
                
            max_value = max(counters)
            max_indices = []
            
            for k in range(vertices_count):
                if (counters[k] == max_value):
                    max_indices.append(k)
            
            winner_idx = max_indices[random.randrange(len(max_indices))]
        
            if current_community in max_indices:
                winner_idx = current_community
                
            if winner_idx != current_community:
                convergence = False
                
            v["communities"] = [winner_idx]
            
    plot_full(graph, output, 1.5)

In [61]:
my_basic_lpa(nba11_20, "all_stars_my_1120.svg")
my_basic_lpa(nba15_20, "all_stars_my_1520.svg")
my_basic_lpa(nba19_20, "all_stars_my_1920.svg")

Iteration 1,communities 83
Iteration 2,communities 22
Iteration 3,communities 16
Iteration 4,communities 11
Iteration 5,communities 7
Iteration 6,communities 4
Plotting graph with 83 vertices and 315 edges, begin...
Plotting sucessful
Iteration 1,communities 60
Iteration 2,communities 18
Iteration 3,communities 13
Iteration 4,communities 12
Plotting graph with 60 vertices and 133 edges, begin...
Plotting sucessful
Iteration 1,communities 39
Iteration 2,communities 16
Iteration 3,communities 16
Plotting graph with 39 vertices and 38 edges, begin...
Plotting sucessful
